<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/SVM_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to read csv file into colaboratory:
!pip install sentence-transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import sentence_transformers
from sentence_transformers import SentenceTransformer, models
data_set=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv')[['event_type', 'cleaned']]
data_set.head(1)

,event_type,cleaned
0,Explosions/Remote violence,regime forces shelled Arnaba town in Idleb cou...


In [4]:
print (f"Shape of original data set is {data_set.shape}")

Shape of original data set is (1034527, 2)


In [5]:
df=data_set.sample(100000).reset_index().drop(['index'], axis = 1)
df.head(1)

,event_type,cleaned
0,Violence against civilians,Saudi border guards reportedly shot and killed...


In [6]:
#del data_set

In [7]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [8]:
train = model.encode(df.cleaned, device='cuda')

In [9]:
from sklearn import preprocessing
# features_trainl encode the target variable
encoder = preprocessing.LabelEncoder()
df["event_type_num"] = encoder.fit_transform(df["event_type"])

In [10]:
df_features = pd.DataFrame()
df_features['cleaned_emb'] = [i for i in train] 
df_features['event_type'] = df["event_type_num"]
df_features.head(3)

,cleaned_emb,event_type
0,"[-0.09404321, 0.37393332, -0.036032546, -0.100...",5
1,"[0.082842916, 0.021187244, -0.0782555, 0.27508...",2
2,"[0.051247198, 0.15182263, 0.04186469, 0.320997...",4


In [11]:
 df_features.to_csv("./features_set.csv")

In [12]:
from sklearn.decomposition import PCA
import time
pca_model = PCA(n_components=50)
start = time.process_time()
pca_model.fit(train)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))
print(time.process_time() - start)

Sum of variance ratios:  0.5972317894773987
18.360749380000016


In [13]:
x_comps = pca_model.transform(train)
x_comps.shape

(100000, 50)

**Model Training**

SVM

In [14]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_comps,df_features['event_type'],test_size=0.2,random_state=42)

In [15]:
from sklearn.svm import SVC
start = time.time() 

svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))

Support Vector Machine Classifier has fitted, this process took 96.96 seconds


In [16]:
svm_classifier.score(x_test,y_test)

0.9248

In [17]:
y_pred = svm_classifier.predict(x_test)

In [18]:
print(y_pred)
print(y_test)

[0 3 0 ... 2 2 0]
75721    0
80184    3
19864    0
76699    0
92991    2
        ..
32595    0
29313    3
37862    2
53421    2
42410    0
Name: event_type, Length: 20000, dtype: int64


In [19]:
from sklearn.metrics import classification_report

print(classification_report(y_true= y_test, y_pred= y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      4837
           1       0.93      0.94      0.93      2514
           2       0.97      0.99      0.98      6986
           3       0.85      0.80      0.82      1670
           4       0.89      0.83      0.86      1082
           5       0.87      0.90      0.88      2911

    accuracy                           0.92     20000
   macro avg       0.91      0.89      0.90     20000
weighted avg       0.92      0.92      0.92     20000



K FOLD

In [20]:
# Making the Confusion Matrix 
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4414  122    1   36   23  241]
 [ 129 2357    1    2   10   15]
 [   2    2 6885   74   13   10]
 [  41    4  192 1329   20   84]
 [  33   28   33   34  897   57]
 [ 136   25    7   88   41 2614]]


In [21]:
# Applying k-Fold Cross Validation 
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = svm_classifier, X = x_train, y = y_train, cv = 10) 
accuracies.mean() 
accuracies.std()

0.00278862085088669